# EDA

## Структура данных

In [ ]:
from glob import glob

pattern = 'data/**/*'
files = glob(pattern, recursive=True)

print(files)

In [ ]:
class EDA:
    pass

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

path = 'data/TIF_original_image_with_mask&geodata.tif'

with rasterio.open(path) as src:
    r = src.read(1).astype(np.int16)
    g = src.read(2).astype(np.int16)
    b = src.read(3).astype(np.int16)

    rgb = np.stack([r, g, b], axis=-1)

    fig = plt.figure(figsize=(10, 15))

    plt.imshow(rgb)
    plt.tight_layout()
    plt.axis('off')
    plt.show()

In [ ]:
import rasterio

path = 'data/RGB_Polazna_cut.tif'

with rasterio.open(path) as src:
    print("✅ Основная информация:")
    print(f"- Ширина: {src.width}")
    print(f"- Высота: {src.height}")
    print(f"- Количество каналов: {src.count}")
    print(f"- Тип данных: {src.dtypes}")  # список по каналам, например ['uint16']
    print(f"- CRS (система координат): {src.crs}")
    print(f"- Transform (геопривязка): {src.transform}")
    print()

    print("🧾 Полные метаданные (src.meta):")
    print(src.meta)
